In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

pd.options.display.max_columns = None

import pickle

In [25]:
# t_seps = [10, 12, 14, 16, 18, 20, 22, 24]
t_seps = [10, 12, 14, 16, 18] # MUST keep the first k values

# trajs = range(2100, 2150, 10)
trajs = sorted([int(f[:4]) for f in os.listdir('./Kaon_results/')]) 
print(trajs)

def mean_err(arr):  # shape: (trajs, ...)
    mean = arr.mean(axis=0)
    err = arr.std(axis=0) / np.sqrt(arr.shape[0])  
    return mean, err  

[1010, 1030, 1040, 1050, 1070, 1080, 1090, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470, 1480, 1490, 1500, 1510, 1520, 1530, 1540, 1550, 1560, 1570, 1580, 1590, 1600, 1610, 1630, 1640, 1650, 1660, 1670, 1680, 1690, 1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790, 1800, 1810, 1820, 1840, 1850, 1860, 1870, 1890, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1990, 2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100, 2110, 2120, 2130, 2140, 2150, 2160, 2170, 2180, 2190, 2200, 2210, 2220, 2230, 2240, 2250]


In [49]:
# # Without disconnected diagrams
# coefficients = {
#     'cs_sBar_d_T1D1': -1/np.sqrt(6), 'cs_sBar_d_T1D2': 2/np.sqrt(6), 
# #     'cs_sBar_d_T2D1': 2/np.sqrt(6), 'cs_sBar_d_T2D2': -2/np.sqrt(6),   # disconnected
    
#     'cs_Hw_T1D1Q1': 1/np.sqrt(6), 'cs_Hw_T1D1Q2': -1/np.sqrt(6), 
#     'cs_Hw_T2D1Q1': 1/np.sqrt(6), 'cs_Hw_T2D1Q2': -1/np.sqrt(6), 
#     'cs_Hw_T2D2Q1': -2/np.sqrt(6), 'cs_Hw_T2D2Q2': 2/np.sqrt(6), 
# #     'cs_Hw_T3D1Q1': -2/np.sqrt(6), 'cs_Hw_T3D1Q2': 2/np.sqrt(6),    # disconnected
# #     'cs_Hw_T3D2Q1': 2/np.sqrt(6), 'cs_Hw_T3D2Q2': -2/np.sqrt(6)   # disconnected
# }

# With disconnected diagrams
coefficients = {
    'cs_sBar_d_T1D1': -1/np.sqrt(6), 'cs_sBar_d_T1D2': 2/np.sqrt(6), 
    
    'cs_sBar_d_T2D1': 2/np.sqrt(6), 'cs_sBar_d_T2D2': -2/np.sqrt(6),   # disconnected
    
    'cs_Hw_T1D1Q1': 1/np.sqrt(6), 'cs_Hw_T1D1Q2': -1/np.sqrt(6), 
    'cs_Hw_T2D1Q1': 1/np.sqrt(6), 'cs_Hw_T2D1Q2': -1/np.sqrt(6), 
    'cs_Hw_T2D2Q1': -2/np.sqrt(6), 'cs_Hw_T2D2Q2': 2/np.sqrt(6), 
    
    'cs_Hw_T3D1Q1': -2/np.sqrt(6), 'cs_Hw_T3D1Q2': 2/np.sqrt(6),    # disconnected
    'cs_Hw_T3D2Q1': 2/np.sqrt(6), 'cs_Hw_T3D2Q2': -2/np.sqrt(6)   # disconnected
}


diagrams = list(coefficients.keys())

Q1_diagrams = [d for d in diagrams if d.endswith('Q1')] 
Q2_diagrams = [d for d in diagrams if d.endswith('Q2')] 
sBar_d_diagrams = [d for d in diagrams if d.startswith('cs_sBar_d')]

print(f'Q1_diagrams: {Q1_diagrams}')
print(f'Q2_diagrams: {Q2_diagrams}')
print(f'sBar_d_diagrams: {sBar_d_diagrams}')

Q1_diagrams: ['cs_Hw_T1D1Q1', 'cs_Hw_T2D1Q1', 'cs_Hw_T2D2Q1', 'cs_Hw_T3D1Q1', 'cs_Hw_T3D2Q1']
Q2_diagrams: ['cs_Hw_T1D1Q2', 'cs_Hw_T2D1Q2', 'cs_Hw_T2D2Q2', 'cs_Hw_T3D1Q2', 'cs_Hw_T3D2Q2']
sBar_d_diagrams: ['cs_sBar_d_T1D1', 'cs_sBar_d_T1D2', 'cs_sBar_d_T2D1', 'cs_sBar_d_T2D2']


In [51]:
def calc_cs(traj):
    data = pickle.load(open(f'./Kaon_results/{traj}.pkl', 'rb'))
    
    eta_Q1_K = sum([data[d] * coefficients[d] for d in Q1_diagrams])
    eta_Q2_K = sum([data[d] * coefficients[d] for d in Q2_diagrams])
    eta_sBar_d_K = sum([data[d] * coefficients[d] for d in sBar_d_diagrams])
    
    eta_Q1_K = eta_Q1_K.real # ??? should I take real part; seems like imag part is small
    eta_Q2_K = eta_Q2_K.real 
    eta_sBar_d_K = eta_sBar_d_K.real
    
    return eta_Q1_K, eta_Q2_K, eta_sBar_d_K # shape (tsep, tx)

def convert_to_dist_from_eta(amp_one_traj): # change to: eta is always at 0 and is on the left
    rst = []
    min_tsep = min(t_seps)
    for i, t_sep in enumerate(t_seps):
        tmp = amp_one_traj[i][t_sep-min_tsep:t_sep+1][::-1]  
        rst.append(tmp)
    rst = np.stack(rst)
    return rst


eta_Q1_K, eta_Q2_K, eta_sBar_d_K = [], [], []
for traj in trajs:
    eta_Q1_K_one_traj, eta_Q2_K_one_traj, eta_sBar_d_K_one_traj = calc_cs(traj)   # shape (tsep, tx)
    eta_Q1_K_one_traj = convert_to_dist_from_eta(eta_Q1_K_one_traj[:len(t_seps)])
    eta_Q2_K_one_traj = convert_to_dist_from_eta(eta_Q2_K_one_traj[:len(t_seps)])
    eta_sBar_d_K_one_traj = convert_to_dist_from_eta(eta_sBar_d_K_one_traj[:len(t_seps)])
    
    eta_Q1_K.append(eta_Q1_K_one_traj)
    eta_Q2_K.append(eta_Q2_K_one_traj)
    eta_sBar_d_K.append(eta_sBar_d_K_one_traj)
    
eta_Q1_K = np.array(eta_Q1_K)  # (trajs, tsep, tx)
eta_Q2_K = np.array(eta_Q2_K)
eta_sBar_d_K = np.array(eta_sBar_d_K)

def jk_c_s(skip_traj_idx=0):

    eta_Q1_K_avg = np.delete(eta_Q1_K, skip_traj_idx, axis=0).mean(axis=0)
    eta_Q2_K_avg = np.delete(eta_Q2_K, skip_traj_idx, axis=0).mean(axis=0)
    eta_sBar_d_K_avg = np.delete(eta_sBar_d_K, skip_traj_idx, axis=0).mean(axis=0)

    c_s_Q1 = -  eta_Q1_K_avg / eta_sBar_d_K_avg
    c_s_Q2 = -  eta_Q2_K_avg / eta_sBar_d_K_avg
    return c_s_Q1, c_s_Q2

c_s_Q1_jk, c_s_Q2_jk = [], []
for skip_traj_idx in range(eta_Q1_K.shape[0]):
    c_s_Q1_jk_one, c_s_Q2_jk_one = jk_c_s(skip_traj_idx)
    c_s_Q1_jk.append(c_s_Q1_jk_one)
    c_s_Q2_jk.append(c_s_Q2_jk_one)
c_s_Q1_jk = np.stack(c_s_Q1_jk)
c_s_Q2_jk = np.stack(c_s_Q2_jk)

c_s_Q1_avg = c_s_Q1_jk.mean(axis=0)
c_s_Q1_err = c_s_Q1_jk.std(axis=0, ddof=0) * np.sqrt(eta_Q1_K.shape[0] - 1)

# eta_Q1_K_avg, eta_Q1_K_err = mean_err(eta_Q1_K)
# eta_Q2_K_avg, eta_Q2_K_err = mean_err(eta_Q2_K)
# eta_sBar_d_K_avg, eta_sBar_d_K_err = mean_err(eta_sBar_d_K)

# c_s_Q1_avg = -  eta_Q1_K_avg / eta_sBar_d_K_avg
# c_s_Q2_avg = -  eta_Q2_K_avg / eta_sBar_d_K_avg
# c_s_Q1_err = np.sqrt(abs(c_s_Q1_avg) * ((eta_Q1_K_err / eta_Q1_K_avg)**2 + (eta_sBar_d_K_err / eta_sBar_d_K_avg)**2) )
# c_s_Q2_err = np.sqrt(abs(c_s_Q2_avg) * ((eta_Q2_K_err / eta_Q2_K_avg)**2 + (eta_sBar_d_K_err / eta_sBar_d_K_avg)**2) )

In [54]:
c_s_Q1_avg

array([[ 0.00107876,  0.00977407,  0.01750707,  0.06389271,  0.00236608,
        -0.09156732, -0.02029644, -0.06796572,  0.02851853,  0.02424301,
         0.00037604],
       [ 0.00076687,  0.00885565,  0.01203897,  0.0235871 ,  0.03447985,
         0.07150502, -0.03169245,  0.00163752,  0.00303522,  0.01643295,
         0.02573638],
       [ 0.00080258,  0.00972558,  0.01075313,  0.01936257,  0.14129367,
         0.00852837,  0.07414852, -0.00976789, -0.04539768,  0.0046616 ,
         0.00136089],
       [ 0.00091018,  0.0086406 ,  0.01052531,  0.01790276,  0.01179827,
         0.01135681,  0.00906703,  0.000947  ,  0.00355861,  0.02504045,
         0.00784132],
       [ 0.00195893,  0.0087743 ,  0.01269822,  0.0186374 ,  0.01729117,
         0.01343708,  0.00709787,  0.00864299,  0.00027273, -0.00367359,
        -0.00202559]])

In [18]:
# def plot_by_tsep(avg, err, title='xxxxx'):
#     fig = go.Figure()
#     for i, t_sep in enumerate(t_seps[:5]):
#         fig.add_trace(
#             go.Scatter(x=list(range(t_sep)), y=avg[i][:t_sep], name=f'tsep={t_sep}',
#                       error_y=dict(type='data', array=err[i][:t_sep]))
#         )
#     fig.update_layout(title_text=title, width=600, height=400)
# #     fig.show()
#     return fig

# plot_by_tsep(eta_Q1_K_avg, eta_Q1_K_err)

In [19]:
c_s_Q1_avg.shape

(5, 11)

In [45]:
weights = 1 / (c_s_Q1_err**2)
weights =  weights / np.nansum(weights, axis=0)

c_s_Q1_avg_avg_tsep = np.nansum(c_s_Q1_avg * weights, axis=0) # average over tsep
c_s_Q1_err_avg_tsep = np.nansum((c_s_Q1_err * weights)**2, axis=0)
c_s_Q2_avg_avg_tsep = np.nansum(c_s_Q2_avg * weights, axis=0) # average over tsep
c_s_Q2_err_avg_tsep = np.nansum((c_s_Q2_err * weights)**2, axis=0)

In [50]:
weights

array([[2.29671702e-01, 1.65010787e-01, 5.60582147e-02, 2.61245375e-04,
        8.00843595e-05, 1.26027741e-04, 2.24412096e-04, 1.52525409e-04,
        3.84140153e-07, 3.00922870e-04, 9.41233451e-01],
       [3.12715490e-01, 2.16410556e-01, 1.90494825e-01, 5.07830899e-02,
        1.23347709e-04, 3.48960225e-05, 1.05481955e-04, 6.10261688e-04,
        4.01899870e-05, 2.20266632e-04, 2.64319208e-04],
       [2.12911246e-01, 2.23390381e-01, 3.08705104e-01, 1.95464072e-01,
        8.41322156e-06, 9.52365405e-04, 1.54923737e-04, 1.79934540e-04,
        2.12227145e-06, 3.94866365e-04, 8.72116935e-03],
       [1.67167353e-01, 2.28094006e-01, 3.16430594e-01, 5.78594243e-01,
        8.43043766e-01, 5.87639316e-01, 2.55493875e-01, 3.72162682e-02,
        5.30438757e-04, 4.01255393e-04, 8.95947342e-03],
       [7.75342083e-02, 1.67094270e-01, 1.28311263e-01, 1.74897349e-01,
        1.56744388e-01, 4.11247395e-01, 7.44021307e-01, 9.61841010e-01,
        9.99426865e-01, 9.98682689e-01, 4.08215870e-

In [46]:
c_s_Q1_avg_avg_tsep

array([ 0.00096249,  0.00913889,  0.01155418,  0.01861727,  0.01266238,
        0.01219875,  0.00760113,  0.0083373 ,  0.0002745 , -0.00364595,
        0.00036018])

In [48]:
def plot_final_cs(cs_avg, cs_err, title='xxxxx'):
    fig = go.Figure()
#     for i, t_sep in enumerate(t_seps[3:]):
    fig.add_trace(
        go.Scatter(x=list(range(len(cs_avg))), y=cs_avg, error_y_array=cs_err)
    )
    fig.update_layout(title_text=title, width=600, height=400,
                     yaxis_title='c_s', xaxis_title='t_x (Distance from Eta)')
    fig.update_layout()
    return fig

plot_final_cs(c_s_Q1_avg_avg_tsep, c_s_Q1_err_avg_tsep, title='c_s Q1').show()
plot_final_cs(c_s_Q2_avg_avg_tsep, c_s_Q2_err_avg_tsep, title='c_s Q2').show()

In [55]:


# plot_by_tsep(c_s_Q1_avg, c_s_Q1_err).show()
# plot_by_tsep(c_s_Q2_avg, c_s_Q2_err).show()

# Old c_s

In [56]:
assert False

AssertionError: 

In [ ]:
c_s_Q1, c_s_Q2 = [], []
for traj in trajs:
    c_s_Q1_one_traj, c_s_Q2_one_traj = calc_cs(traj)   # shape (tsep, tx)
    c_s_Q1.append(c_s_Q1_one_traj)
    c_s_Q2.append(c_s_Q2_one_traj)
c_s_Q1 = np.array(c_s_Q1)  # (trajs, tsep, tx)
c_s_Q2 = np.array(c_s_Q2)

c_s_Q1_avg, c_s_Q1_err = mean_err(c_s_Q1)
c_s_Q2_avg, c_s_Q2_err = mean_err(c_s_Q2)

In [ ]:
# zyd: spikes are probably because that denominator is small at these positions
# Result is messy; but result is very good if we comment out the disconnected diagrams

fig = go.Figure()
for i, t_sep in enumerate(t_seps[:5]):
    fig.add_trace(
        go.Scatter(x=list(range(t_sep)), y=c_s_Q1_avg[i][:t_sep], name=f'tsep={t_sep}',
                  error_y=dict(type='data', array=c_s_Q1_err[i][:t_sep]))
    )
fig.update_layout(title_text='c_s for Q1', width=600, height=400)
fig.show()

fig = go.Figure()
for i, t_sep in enumerate(t_seps[:5]):
    fig.add_trace(
        go.Scatter(x=list(range(t_sep)), y=c_s_Q2_avg[i][:t_sep], name=f'tsep={t_sep}',
                  error_y=dict(type='data', array=c_s_Q2_err[i][:t_sep]))
    )
fig.update_layout(title_text='c_s for Q2', width=600, height=400)
fig.show()

In [ ]:
# c_s_Q1, c_s_Q2 = calc_cs(2100)

# fig = go.Figure()
# for i, t_sep in enumerate(t_seps[:5]):
#     fig.add_trace(go.Scatter(x=list(range(t_sep)), y=c_s_Q1[i][:t_sep], name=f'tsep={t_sep}'))
# fig.update_layout(title_text='c_s for Q1', width=600, height=400)
# fig.show()


# fig = go.Figure()
# for i, t_sep in enumerate(t_seps[:5]):
#     fig.add_trace(go.Scatter(x=list(range(t_sep)), y=c_s_Q2[i][:t_sep], name=f'tsep={t_sep}'))
# fig.update_layout(title_text='c_s for Q2', width=600, height=400)
# fig.show()